Strategy Optimization

In [84]:
import SMABacktester as SMA
import MeanRevBacktester as MeanRev
import pandas as pd
import numpy as np

In [86]:
df = pd.read_csv("twenty_minutes.csv", parse_dates=["time"], index_col="time")
symbol = "EURUSD"
symbol

'EURUSD'

In [88]:
def optimal_strategy(parameters):
    start = "2019-01-01"
    end = "2020-08-30"
    symbol = "EURUSD"
    tc = 0.000059

    #SMA
    tester1 = SMA.SMABacktester(symbol, int(parameters[0]), int(parameters[1]), start, end, tc)
    tester1.test_strategy()

    #Bollinger
    tester2 = MeanRev.MeanRevBacktester(symbol, int(parameters[2]), int(parameters[3]),  start, end, tc)
    tester2.test_strategy()

    # Create Comb
    comb = tester1.results.loc[:, ["returns", "position"]].copy()
    comb.rename(columns = {"position":"position_SMA"}, inplace = True)
    comb["position_MR"] = tester2.results.position

    # Methode 2
    comb["position_comb"] = np.sign(comb.position_MR + comb.position_SMA)

    # Busy Hours
    comb["NYTime"] = comb.index.tz_convert("America/New_York")
    comb["hour"] = comb.NYTime.dt.hour
    comb.position_comb = np.where(comb.hour.between(2, 12), comb.position_comb, 0)

    # Backtest
    comb["strategy"] = comb["position_comb"].shift(1)*comb["returns"]
    comb.dropna(inplace = True)
    comb["trades"] = comb.position_comb.diff().fillna(0).abs()
    comb.strategy = comb.strategy - comb.trades - tc
    comb["creturns"] = comb["returns"].cumsum().apply(np.exp)
    comb["cstrategy"] = comb["strategy"].cumsum().apply(np.exp)

    return -comb["cstrategy"].iloc[-1]

In [90]:
optimal_strategy((50, 150, 75, 3))

-0.0

In [92]:
from scipy.optimize import minimize

In [94]:
binds = ((25, 75), (100, 200), (50, 100), (1, 5))
binds

((25, 75), (100, 200), (50, 100), (1, 5))

In [43]:
star_par = (50, 150, 75, 3)